In [25]:
import pandas as pd
import os
filepath = os.path.abspath('')
samplesFilepath = filepath + '/data-from-samples.csv'
calibrationFilepath = filepath + '/calibration-data.csv'

In [26]:
samplesData = pd.read_csv(samplesFilepath)
calibrationData = pd.read_csv(calibrationFilepath)
print(calibrationData)

   Current  Voltage   High    Low
0     1.50      0.4 -0.643  0.449
1     0.00      0.0  0.003    NaN
2    -1.50      0.4  0.652 -0.533
3     1.25      0.4 -0.537  0.336
4    -1.25      0.3  0.549 -0.382
5     1.75      0.4 -0.756  0.487
6    -1.75      0.4  0.764 -0.447
7     2.00      0.5 -0.865  0.444
8    -2.00      0.5  0.877 -0.486


In [30]:
calibrationData = calibrationData.sort_values('Current')
print(calibrationData)

   Current  Voltage   High    Low
8    -2.00      0.5  0.877 -0.486
6    -1.75      0.4  0.764 -0.447
2    -1.50      0.4  0.652 -0.533
4    -1.25      0.3  0.549 -0.382
1     0.00      0.0  0.003    NaN
3     1.25      0.4 -0.537  0.336
0     1.50      0.4 -0.643  0.449
5     1.75      0.4 -0.756  0.487
7     2.00      0.5 -0.865  0.444
